In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from scipy import stats
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from statsmodels.sandbox.stats.multicomp import TukeyHSDResults
import pickle
from sklearn.metrics import confusion_matrix

In [2]:
def Henon(n):
    a = 1.4
    b = 0.3
    x = np.zeros((n+1))
    y = np.zeros((n+1))
    for i in range(n):
        x[i+1] = 1-a*x[i]*x[i] + y[i]
        y[i+1] = b*x[i]
    return (x,y)

In [3]:
def Ventanas(x,size):
    size += 1
    arr = np.empty((0,size))
    for i in range(len(x) - size):
        arr = np.vstack((arr,x[i:i + size]))
    return (arr[:,:-1],arr[:,-1])

In [ ]:
Xinicial,_ = Henon(1000)
X,y = Ventanas(Xinicial,25)
#X,y = Ventanas(Xinicial,50)

## Entrenamiento de Modelos

**kFold - Support Vector Regression**

In [ ]:
def SVR_multiple():
    
    global resultados_train
    global resultados_test
    global res_train
    global res_test
    global model
    global train_indices
    global test_indices
    global etiquetas
    
    etiquetas.extend(['SVR linear C1',
                 'SVR poly 2º C1',
                 'SVR poly 3º C1',
                 'SVR poly 4º C1',
                 'SVR rfb auto C1',
                 'SVR rfb 0.05 C1',
                 'SVR rfb 0.1 C1',
                 'SVR rfb 0.2 C1'])
    etiquetas.extend(['SVR linear C100',
                 'SVR poly 2º C100',
                 'SVR poly 3º C100',
                 'SVR poly 4º C100',
                 'SVR rfb auto C100',
                 'SVR rfb 0.05 C100',
                 'SVR rfb 0.1 C100',
                 'SVR rfb 0.2 C100'])
    dicc = [{'C': 1.0, 'kernel': 'linear', 'degree': 1, 'gamma': 'auto', 'tol': 0.1},
            {'C': 1.0, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto', 'tol': 0.1},
            {'C': 1.0, 'kernel': 'poly', 'degree': 3, 'gamma': 'auto', 'tol': 0.1},
            {'C': 1.0, 'kernel': 'poly', 'degree': 4, 'gamma': 'auto', 'tol': 0.1},
            {'C': 1.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 'auto', 'tol': 0.1},
            {'C': 1.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.05, 'tol': 0.1},
            {'C': 1.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.1, 'tol': 0.1},
            {'C': 1.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.2, 'tol': 0.1},
            {'C': 100.0, 'kernel': 'linear', 'degree': 1, 'gamma': 'auto', 'tol': 0.1},
            {'C': 100.0, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto', 'tol': 0.1},
            {'C': 100.0, 'kernel': 'poly', 'degree': 3, 'gamma': 'auto', 'tol': 0.1},
            {'C': 100.0, 'kernel': 'poly', 'degree': 4, 'gamma': 'auto', 'tol': 0.1},
            {'C': 100.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 'auto', 'tol': 0.1},
            {'C': 100.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.05, 'tol': 0.1},
            {'C': 100.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.1, 'tol': 0.1},
            {'C': 100.0, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.2, 'tol': 0.1}]

    for i in range(len(dicc)):
        train_indices2 = []
        test_indices2 = []
        for j in range(3):
            for train_index, test_index in kf.split(X):
                X_train, X_test,y_train, y_test = X[train_index],X[test_index],y[train_index],y[test_index]
                train_indices2.append(train_index)
                test_indices2.append(test_index)
                alg = SVR(**dicc[i])
                alg.fit(X_train, y_train)
                model.append(alg)
                y_pred_test = alg.predict(X_test)
                y_pred_train = alg.predict(X_train)
                resultados_train.append(mean_squared_error(y_train,y_pred_train))
                resultados_test.append(mean_squared_error(y_test,y_pred_test))
        res_train = np.hstack((res_train, np.array(resultados_train,ndmin=2).T))
        res_test = np.hstack((res_test, np.array(resultados_test,ndmin=2).T))
        resultados_train = []
        resultados_test = []
        test_indices.append(test_indices2)

**kFold - Multi-layer Perceptron Regressor**

In [ ]:
def MLP_multiple():
    
    global resultados_train
    global resultados_test
    global res_train
    global res_test
    global model
    global train_indices
    global test_indices
    global etiquetas
    
    etiquetas.extend(['MLPR sgd 10 relu',
                 'MLPR sgd 30 relu',
                 'MLPR sgd 50 relu',
                 'MLPR adam 10 relu',
                 'MLPR adam 30 relu',
                 'MLPR adam 50 relu'])
    etiquetas.extend(['MLPR sgd 10 tanh',
                 'MLPR sgd 30 tanh',
                 'MLPR sgd 50 tanh',
                 'MLPR adam 10 tanh',
                 'MLPR adam 30 tanh',
                 'MLPR adam 50 tanh'])
    dicc = [{'hidden_layer_sizes': (10,), 'solver': 'sgd',  'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'sgd',  'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'sgd',  'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'adam', 'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'adam', 'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'adam', 'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'sgd',  'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'sgd',  'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'sgd',  'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'adam', 'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'adam', 'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'adam', 'activation': 'tanh','early_stopping': True, 'max_iter': 1500}]

    for i in range(len(dicc)):
        train_indices2 = []
        test_indices2 = []
        for j in range(3):
            for train_index, test_index in kf.split(X):
                X_train, X_test,y_train, y_test = X[train_index],X[test_index],y[train_index],y[test_index]
                train_indices2.append(train_index)
                test_indices2.append(test_index)
                alg = MLPRegressor(**dicc[i])
                alg.fit(X_train, y_train)
                model.append(alg)
                y_pred_test = alg.predict(X_test)
                y_pred_train = alg.predict(X_train)
                resultados_train.append(mean_squared_error(y_train,y_pred_train))
                resultados_test.append(mean_squared_error(y_test,y_pred_test))
        res_train = np.hstack((res_train, np.array(resultados_train,ndmin=2).T))
        res_test = np.hstack((res_test, np.array(resultados_test,ndmin=2).T))
        resultados_train = []
        resultados_test = []
        train_indices.append(train_indices2)
        test_indices.append(test_indices2)

In [ ]:
#definicion de variables globales.
kf = KFold(n_splits=10,shuffle=True)
resultados_train = []
resultados_test = []
model = []
res_train = np.empty((30,0))
res_test = np.empty((30,0))
train_indices = []
test_indices = []
etiquetas = []
scaler = StandardScaler()

Xinicial,_ = Henon(1000)

for t_ventana in (25,50):
    X,y = Ventanas(Xinicial, t_ventana)
    X = scaler.fit_transform(X)
    y = (y - np.mean(y))/np.std(y)

    %time SVR_multiple()
    %time MLP_multiple()


**Persistencia**

In [ ]:
with open('p4_henon-res_test.bin','wb') as file:
    pickle.dump(res_test, file)
with open('p4_henon-train_indices.bin','wb') as file:
    pickle.dump(train_indices, file)
with open('p4_henon-test_indices.bin','wb') as file:
    pickle.dump(test_indices, file)
with open('p4_henon-models.bin','wb') as file:
    pickle.dump(model, file)

In [ ]:
# with open('p4_henon-res_test.bin','rb') as file:
#     res_test = pickle.load(file)
# with open('p4_henon-train_indices.bin','wb') as file:
#     train_indices = pickle.load(file)
# with open('p4_henon-test_indices.bin','wb') as file:
#     test_indices = pickle.load(file)
# with open('p4_henon-models.bin','wb') as file:
#     model = pickle.load(file)

**Test de normalidad**  
Rechazamos normalidad, realizamos el test no paramétrico de Kruskal-Wallis en lugar de un Anova 

In [ ]:
np.apply_along_axis(lambda x: stats.shapiro(x)[1], axis=0, arr=res_test)

**Test de Kruskal-Wallis**  
Rechazamos que las precisiones sean similares

In [ ]:
print(stats.kruskal(*zip(*list(res_test))))

In [ ]:
labels = [etiquetas[i] for i in range(res_test.shape[1]) for _ in range(res_test.shape[0])]
arr_test = res_test.flatten('F')
resultados = pairwise_tukeyhsd(arr_test,labels)
# resultados.summary()

In [ ]:
#seleccionamos el que tiene mejor media y vemos en el anterior cuales son iguales
best_group = np.argmin(np.mean(res_test, axis = 0))
best_group = etiquetas[int(best_group)]

In [ ]:
print(resultados.plot_simultaneous(comparison_name=best_group))

In [ ]:
resultados = resultados._results_table.data

In [ ]:
group1 = resultados[0].index('group1')
group2 = resultados[0].index('group2')
reject = resultados[0].index('reject')
best_groups = []
for row in resultados[1:]:
    if (row[group1] == str(best_group) or row[group2] == str(best_group)) and not row[reject]:
        best_groups.append(row[group1])
        best_groups.append(row[group2])
best_groups.append(best_group)
best_groups = set(best_groups)
sorted(best_groups)

**Nos quedamos con el más sencillo**  
El cual asumimos que es el MLPR con:
- sgd (Stochastic Gradient Descent)  
- relu (Rectified linear unit function)  
- 10 neuronas en la capa oculta

In [ ]:
for ind, tag in enumerate(best_groups):
    mean_test = np.mean(res_test[:,ind])
    std_test = np.std(res_test[:,ind])
    print('{}: {:.2f} +- {:.2f}'.format(tag, mean_test, std_test))